In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key=os.getenv("OPENAI_API_KEY")
print("API Key is Loaded: ", openai_api_key is not None)

API Key is Loaded:  True


In [2]:
from crewai.llm import LLM

In [3]:
llm=LLM(
    model="gpt-5.1",
    temperature=0.1
)
response=llm.call("What is Crew AI?")
print(response)

“Crew AI” isn’t a single, universally defined product name; it’s used in a few different contexts. The most common current reference is:

**CrewAI (open‑source multi‑agent framework)**  
CrewAI is an open‑source Python framework that lets you build “crews” of AI agents that work together to complete complex tasks. Key ideas:

- **Multiple agents**: You define several specialized agents (e.g., “researcher,” “writer,” “planner”) instead of one monolithic assistant.  
- **Roles and tools**: Each agent can have a role, goals, and access to tools (APIs, databases, code execution, etc.).  
- **Orchestration**: The framework coordinates how agents talk to each other, pass results, and decide next steps.  
- **Use cases**: Content generation pipelines, research assistants, workflow automation, coding helpers, and other multi-step processes.

It’s similar in spirit to other “multi‑agent” or “AI workflow” systems (like LangGraph, AutoGen, etc.), but with a focus on defining a “crew” that collabo

In [5]:
from crewai.tools import BaseTool

class AITool(BaseTool):
    name: str = "Abbreviation Detector"
    description: str = "Detect abbreviation and suggest replacement the clear communications"

    def _run(self, email:str) -> str:
        replacements = {
          "ASAP": "As soon as possible",
          "WIP": "Work in progress",
          "POC": "Proof of concept",
          "TBD":"To be decide",
          "FYI":"For your information",
          "ETA":"Estimiated time arrival",
          "EOD":"End of the day"
        }
        email_lower=email.lower()
        suggestions=[
            f"Consider replacing '{abbr}' with '{full_form}'."
            for abbr, full_form in replacements.items()
            if abbr.lower() in email_lower
        ]
        return "\n".join(suggestions) if suggestions else "No Abbreviation detected."

ai_tool=AITool()

In [7]:
from crewai import Agent, Task, Crew

AI_Email_Writing_Expert=Agent(
    role="AI Email Writing Expert",
    goal="Enhance email content to ensure it is professional, concise and effectively communicates the intended message",
    backstory="A polished communcation strategist dedicated to enhancing email quality for better engagement and professionalism",
    verbose=True,
    tools=[ai_tool],
    llm=llm
)

original_email="""
Hi team, FYI, just want to following up with you that the deadline of the projects POC WIP is coming soon, so where we are 
in the work, TBD try to complete ASAP.
Let me know if you facing any blocker in the work EOD.
Thanki you
"""

improve_email_task=Task(
    description=f"""Rewrite the following email to make it more professional and engaging
    Expand abbreviations:
    '''{original_email}'''""",
    agent=AI_Email_Writing_Expert,
    expected_output="A well-structured, professionally written email with clear formatting and concise content,",
    async_execution=False,
)
crew=Crew(
    agents=[AI_Email_Writing_Expert],
    tasks=[improve_email_task],
    verbose=True
)

result=crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1a5c6e38-3c6a-413b-97c5-45f7958a4d4e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: AI Email Writing Expert                                                                                 │
│                                                                                                                 │
│  Task: Rewrite the following email to make it more professional and engaging                                    │
│      Expand abbreviations:                                                                                      │
│      '''                                                                                                        │
│  Hi team, FYI, just want to following up with you that the deadline of the projects POC WIP is coming soon, so  │
│  where we are                                                                                                   │
│  in the work, TBD try to complete ASAP.                                                                         │
│  Let me know if you facing any blocker in the work EOD.                                                         │
│  Thanki you                                                                                                     │
│  '''                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: AI Email Writing Expert                                                                                 │
│                                                                                                                 │
│  Thought: Action: Abbreviation Detector                                                                         │
│                                                                                                                 │
│  Using Tool: Abbreviation Detector                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: AI Email Writing Expert                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Upcoming Deadline for Proof of Concept Work in Progress                                               │
│                                                                                                                 │
│  Hi team,                                                                                                       │
│                                                                                                                 │
│  For your information, I would like to follow up regarding the upcoming deadline for the projects’ Proof of     │
│  Concept work in progress.                                                                                      │
│                                                                                                                 │
│  Please provide a quick update on your current status and aim to complete your tasks as soon as possible.       │
│                                                                                                                 │
│  If you are facing any blockers, let me know by the end of the day so we can address them promptly.             │
│                                                                                                                 │
│  Thank you,                                                                                                     │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1a5c6e38-3c6a-413b-97c5-45f7958a4d4e                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Upcoming Deadline for Proof of Concept Work in Progress                                 │
│                                                                                                                 │
│  Hi team,                                                                                                       │
│                                                                                                                 │
│  For your information, I would like to follow up regarding the upcoming deadline for the projects’ Proof of     │
│  Concept work in progress.                                                                                      │
│                                                                                                                 │
│  Please provide a quick update on your current status and aim to complete your tasks as soon as possible.       │
│                                                                                                                 │
│  If you are facing any blockers, let me know by the end of the day so we can address them promptly.             │
│                                                                                                                 │
│  Thank you,                                                                                                     │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 81fe561a-bf7a-4c1f-a69b-a14f3860bbce                                                                     │
│  Agent: AI Email Writing Expert                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RecursionError: maximum recursion depth exceeded